# State Construction Task

In [5]:
from datetime import datetime
import itertools
import io

import numpy as np
import scipy.sparse as sp
import tensorflow as tf

from tqdm import tqdm
import matplotlib.pyplot as plt

generate a parametric hamiltonian with `tf.Variable`

In [16]:
def generate_hamiltonain(dim):
    def generate_parametric(n, dv_lambda):
        params = tf.cast(tf.Variable(tf.random_normal((n, ))), 'complex128')
        places = tf.constant(np.stack([dv_lambda(c) for c in range(n)]), dtype='complex128')
        return tf.tensordot(places, params, axes=[0, 0])
    

    neardiag = lambda i : sp.coo_matrix(
        ([1, 1], ([i, (i + 1) % dim], [(i + 1) % dim, i % dim])), shape=(dim, dim)).toarray()

    diag = lambda i : sp.coo_matrix(([1], ([i], [i])), shape=(dim, dim)).toarray()
    h = generate_parametric(dim - 1, neardiag) + generate_parametric(dim, diag)
    
    return h

implement the optimization problem as a computation graph

In [17]:
def generate_graph(dim):
    source = tf.constant(np.array([1] + [0] * (dim - 1)), dtype='complex128')
    target_vec = np.random.normal(size=(dim,)) + 1j * np.random.normal(size=(dim,)) 
    target_vec /= np.linalg.norm(target_vec)
    target= tf.constant(target_vec, dtype='complex128')

    h = generate_hamiltonain(dim)
    gate = tf.linalg.expm(1j * h)
    estimated_target = tf.matmul(gate, source)

    target_dot = tf.matmul(target, estimated_target, adjoint_a=True)
    fidelity = tf.abs(target_dot)
    loss = 1 - fidelity

    summary_op = tf.summary.scalar('fidelity', fidelity)
    optimize_op = tf.train.MomentumOptimizer(momentum=0.8,learning_rate=0.5).minimize(loss)
    return optimize_op, fidelity

storing metrics (using tensorboard)

In [18]:
def plot_matrix(h):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(h, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Matrix")
    plt.colorbar()

    threshold = h.max() / 2.
    for i, j in itertools.product(range(h.shape[0]), range(h.shape[1])):
        color = "white" if h[i, j] > threshold else "black"
        plt.text(j, i, h[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    return figure

def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

run the whole

In [19]:
folder = './tensorboard_logs/%s' % datetime.now().isoformat()
dim = 8
fs = []
tf.reset_default_graph()
with tf.Session() as session:
    for _ in range(100):
        optimize_op, fidelity_op = generate_graph(dim)
        init_op = tf.global_variables_initializer()
        session.run(init_op)
        for i in tqdm(range(5000)):
            _, f = session.run([optimize_op, fidelity_op])
        fs.append(f)

W0813 18:43:48.471845 140050748647040 deprecation.py:323] From /home/sajad/Works/Quantum/quantum-state-construction/venv/lib/python3.7/site-packages/tensorflow/python/ops/linalg/linalg_impl.py:280: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ValueError: Shape must be rank 2 but is rank 1 for 'MatMul' (op: 'MatMul') with input shapes: [8,8], [8].